## 初始化

In [1]:
# 初始化
import django_setup
import nest_asyncio
nest_asyncio.apply()
# 添加cache 
# 以下要求NotebookCacheManager.py 和 cache目录都在 当前测试文件的相同目录下（在notebook目录下）
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")

from pprint import pprint

# cache使用示例： 
# cache.save('存储文件名', 参数)，  a_loaded = cache.load('存储文件名')， 存在pkl文件下，点开为乱码
# cache.save_json, cache.load_json, 存在json文件下，点开可读
# cache.save_md, cache.load_md，存在md文件下，点开可读
# cache.save_json('outline1/test1') 可构建二级目录

In [ ]:
# 招标文件提取内容 准备就绪
from apps.projects.models import Project, Task, TaskType
from pprint import pprint
project = Project.objects.get(project_name='test1')
task = Task.objects.get(stage__project=project, type=TaskType.OUTLINE_ANALYSIS_TASK)
stage = task.stage
pprint(project.tender_file_extraction)
cache.save_json('招标文件/tender_file_extraction', project.tender_file_extraction)

## PART-1. 招标文件 大纲分析


### 1.1 分析第一层级标题
输入： tender_file_extraction
输出： title_lists_H1， tender_file_extraction_H1


In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction = cache.load_json('招标文件/tender_file_extraction')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction)
print(headings)

In [ ]:
# LLM 数据准备
# 核心功能模块
# 输入： , 输出：model_params, task, meta, formatted_message
# 核心功能模块： TendeorOutlinesL1 里的 prepare_context 调用 TiptapUtils.extract_indexed_paragraphs
from apps.projects.services.prompts.structuring.tender_outlines_L1 import TenderOutlinesL1PromptBuilder
# #获取LLM分析相关数据
prompt_builder = TenderOutlinesL1PromptBuilder(tender_file_extraction)
prompt_config, task_inputs, meta = prompt_builder.output_params()
# # 模拟的prompt, 
raw_prompts, formatted_prompts = prompt_builder.simulate_prompt()
# 检查输出类型
prompt_builder.debug_output()

cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/prompt_config', prompt_config)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/task_inputs', task_inputs)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/meta', meta)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/formatted_prompts', formatted_prompts)


In [ ]:
# LLM分析
# 输入：model_params, task_l1,  输出：针对tender_file_extraction的H1层级标题列表 （json格式）
from apps.projects.services.llm.llm_client import LLMClient

analyzer = LLMClient(prompt_config)
# 大模型输出的是json str格式，可能带有标记噪音
results = await analyzer.process_with_limit(task_inputs, limit=5)


In [ ]:
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/raw_results', results)
print(type(results), len(results), type(results[0]))

In [ ]:
# 清除标记，转为json dict格式
from apps.projects.services.llm.llm_output_processor import LLMOutputProcessor
clean_parsed_results = LLMOutputProcessor().merge_outputs(results)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/clean_parsed_results', clean_parsed_results)
print(type(clean_parsed_results), len(clean_parsed_results), type(clean_parsed_results[0]))


In [ ]:
# 更新文档标题
# 输入： doc, title_lits, index_path_map， 输出：doc_H1
# 核心功能组件： Tiptap.helpers.TiptapUtils.update_titles_from_list()
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_H1 = TiptapUtils.update_titles_from_list(
    doc=tender_file_extraction, 
    title_list=clean_parsed_results, 
    index_path_map=meta["index_path_map"]
    )

cache.save_json('招标文件/tender_file_extraction_H1', tender_file_extraction_H1)
# 检查标题是否已经改到内容里去了？
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.find_all_headings(tender_file_extraction_H1)
print(TiptapUtils.print_headings(tender_file_extraction_H1))

### 1.2 分析第二、三层级标题
输入： tender_file_extraction_H1， 输出：title_lists_H2/3, tender_file_extraction_H23


In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_H1 =cache.load_json('招标文件/tender_file_extraction_H1')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction_H1)
print(headings)


In [2]:
# LLM 数据准备
# 输入： tender_file_extraction_H1, 输出：model_params, task, meta, formatted_message
# 核心功能模块： TenderOutlinesL2 里 prepare_context 调用 TiptapUtils.extract_chapters， 
from apps.projects.services.prompts.structuring import TenderOutlinesL2PromptBuilder
# #获取LLM分析相关数据
prompt_builder = TenderOutlinesL2PromptBuilder(tender_file_extraction_H1)
prompt_config, task_inputs, meta = prompt_builder.output_params()
# # 模拟的prompt, 
raw_prompts, formatted_prompts = prompt_builder.simulate_prompt()

cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/prompt_config', prompt_config)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/task_inputs', task_inputs)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/meta', meta)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/formatted_prompts', formatted_prompts)

In [ ]:
# LLM batch分析
from apps.projects.services.llm.llm_client import LLMClient
from apps.projects.services.llm.llm_output_processor import LLMOutputProcessor
analyzer = LLMClient(prompt_config)
raw_results = await analyzer.process_with_limit(task_inputs, limit=5)

clean_parsed_results = LLMOutputProcessor().merge_outputs(raw_results)
print(f'result类型： {type(raw_results)}, cleaned_parsed_result类型：{type(clean_parsed_results)}')

# 存储OUTLINE_L2 结果
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/raw_results', raw_results)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/clean_parsed_results', clean_parsed_results)

In [ ]:
# LLM 结果修正
# 将所有的'level'值加1
for item in clean_parsed_results:
    if 'level' in item:
        item['level'] += 1

from pprint import pprint
pprint(clean_parsed_results)

In [ ]:
# 更新文档标题 H2-3
# 输入： doc, title_lits, index_path_map， 输出：doc_H1
# 核心功能组件： Tiptap.helpers.TiptapUtils.update_titles_from_list()
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_H2 = TiptapUtils.update_titles_from_list(
    doc=tender_file_extraction_H1, 
    title_list=clean_parsed_results, 
    index_path_map=meta["index_path_map"]
    )

# 检查标题是否已经改到内容里去了？
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.find_all_headings(tender_file_extraction_H2)
from pprint import pprint
#pprint(headings)
print(TiptapUtils.print_headings(tender_file_extraction_H2))

# 存储tender_file_extraction_L2 结果
cache.save_json('招标文件/tender_file_extraction_H2', tender_file_extraction_H2)

### 1.3 添加前言目录
输入： tender_file_extraction   输出: tender_file_extraction_final

In [1]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_H2 =cache.load_json('招标文件/tender_file_extraction_H2')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction_H2)
print(headings)

[H1] 第一章 招标公告
  [H2] 项目编号
  [H2] 采购组织类型
  [H2] 采购方式
  [H2] 招标最高限价
  [H2] 采购内容、数量及简要技术要求
  [H2] 合格投标人的资格要求(本项目采用资格后审)
  [H2] 招标公告的发布
  [H2] 招标文件的发售
  [H2] 投标保证金
  [H2] 投标截止时间和地点
  [H2] 开标时间及地点
  [H2] 评标办法
  [H2] 业务咨询
[H1] 第二章 招标需求
  [H2] 一、概述
  [H2] 二、采购内容
  [H2] 三、技术和质量要求
    [H3] 2.花生油主要技术参数
    [H3] 4.样品
    [H3] *5. 送达地点
    [H3] *6. 交货时间
  [H2] 四、履约保证金
[H1] 第三章 投标人须知
  [H2] 一、总则
    [H3] （一）适用范围
    [H3] （二） 定义
    [H3] （三） 招标方式
    [H3] （四）投标委托
    [H3] （五） 踏勘现场（本项目不适用）和投标费用
    [H3] （六） 联合体投标
    [H3] （七）转包
    [H3] （八）、特别说明：
    [H3] （九） 质疑和投诉
  [H2] 二、招标文件
    [H3] （一）招标文件的构成。本招标文件由以下部分组成：
    [H3] （二） 投标人的风险
    [H3] （三） 招标文件的澄清与修改
    [H3] （四）、对招标文件的异议
  [H2] 三、投标文件的编制
    [H3] （一）、投标文件的组成
    [H3] （二）、投标文件的语言及计量
    [H3] （三）、投标报价
    [H3] （四）、投标文件的有效期
    [H3] （五）、投标保证金
    [H3] （六）、投标文件的签署和份数
    [H3] （七）、投标文件的包装、递交、修改和撤回
    [H3] （八）、投标无效的情形
  [H2] 四、开标
    [H3] （一）、开标准备
    [H3] （二）、开标程序：
  [H2] 五、评标
    [H3] （一）、组建评标委员会
    [H3] （二）、评标的方式
    [H3] （三）、评标程序
    [H3] （四）、澄

In [2]:
# 直接添加前言
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_final = TiptapUtils.add_introduction_headings(tender_file_extraction_H2)
pprint(tender_file_extraction_final)
cache.save_json('招标文件/tender_file_extraction_final', tender_file_extraction_final)

{'content': [{'attrs': {'text_align': 'left'},
              'content': [{'marks': [{'type': 'bold'}],
                           'text': '北京京铁运恒采购供应站有限公司',
                           'type': 'text'},
                          {'text': ' ', 'type': 'text'},
                          {'marks': [{'type': 'bold'}],
                           'text': '2024',
                           'type': 'text'},
                          {'text': ' ', 'type': 'text'},
                          {'marks': [{'type': 'bold'}],
                           'text': '年端午节物资采购项目',
                           'type': 'text'}],
              'type': 'paragraph'},
             {'attrs': {'text_align': 'left'},
              'content': [{'marks': [{'type': 'bold'}],
                           'text': '招标文件',
                           'type': 'text'}],
              'type': 'paragraph'},
             {'attrs': {'text_align': 'left'},
              'content': [{'marks': [{'type': 'bold'}],
                          

In [3]:
#打印 带前言的文档大纲框架 
from apps.projects.tiptap.helpers import TiptapUtils
enhanced_toc=TiptapUtils.print_enhanced_toc(tender_file_extraction_final)
print(enhanced_toc)

[H1] 第一章 招标公告 (path:[45])
  [H2] 项目编号 (path:[47])
  [H2] 采购组织类型 (path:[48])
  [H2] 采购方式 (path:[49])
  [H2] 招标最高限价 (path:[50])
  [H2] 采购内容、数量及简要技术要求 (path:[51])
  [H2] 合格投标人的资格要求(本项目采用资格后审) (path:[53])
  [H2] 招标公告的发布 (path:[65])
  [H2] 招标文件的发售 (path:[67])
  [H2] 投标保证金 (path:[75])
  [H2] 投标截止时间和地点 (path:[78])
  [H2] 开标时间及地点 (path:[82])
  [H2] 评标办法 (path:[85])
  [H2] 业务咨询 (path:[86])
[H1] 第二章 招标需求 (path:[94])
  [H2] 前言 (path:[95])
  [H2] 一、概述 (path:[96])
  [H2] 二、采购内容 (path:[102])
  [H2] 三、技术和质量要求 (path:[104])
    [H3] 2.花生油主要技术参数 (path:[106])
    [H3] 4.样品 (path:[111])
    [H3] *5. 送达地点 (path:[117])
    [H3] *6. 交货时间 (path:[119])
  [H2] 四、履约保证金 (path:[121])
[H1] 第三章 投标人须知 (path:[124])
  [H2] 一、总则 (path:[130])
    [H3] 前言 (path:[131])
    [H3] （一）适用范围 (path:[132])
    [H3] （二） 定义 (path:[134])
    [H3] （三） 招标方式 (path:[140])
    [H3] （四）投标委托 (path:[142])
    [H3] （五） 踏勘现场（本项目不适用）和投标费用 (path:[144])
    [H3] （六） 联合体投标 (path:[150])
    [H3] （七）转包 (path:[152])
    [H3] （八）、特别说明： (path:[154])
  

### 1.4 增强目录分析（图表）

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final =cache.load_json('招标文件/tender_file_extraction_final')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction_final)
print(headings)

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final, 输出：model_params, task, meta, formatted_message
# 核心功能模块： TenderOutlinesL2 里 prepare_context 调用 TiptapUtils.extract_chapters， 
from apps.projects.services.tasks_preparation.tender_outlines_tables import TenderOutlinesTables
# #获取LLM分析相关数据
data_cooker_tb = TenderOutlinesTables(tender_file_extraction_final)
model_params_tb, tasks_tb, meta_tb = data_cooker_tb.output_params()
# # 模拟的prompt, 
simulated_prompts_tb, formatted_messages_tb = data_cooker_tb.simulate_prompt()

cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/model_params_tb', model_params_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/tasks_tb', tasks_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/meta_tb', meta_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/formatted_messages_tb', formatted_messages_tb)

In [ ]:
# LLM batch分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import process_concurrent_JSON_outputs
analyzer = LLMService(model_params_tb)
results_tb = await analyzer.process_with_limit(tasks_tb, limit=5)
pprint(results_tb)

results_tb_clean_parsed = process_concurrent_JSON_outputs(results_tb)
print(f'result类型： {type(results_tb)}, cleaned_result类型：{type(results_tb_clean_parsed)}')

# 存储OUTLINE_L2 结果
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/results', results_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/results_clean_parsed', results_tb_clean_parsed)

In [ ]:
# 更新文档，给节点添加caption note
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_final_tb = TiptapUtils.add_captions_to_nodes(
    doc = tender_file_extraction_final, 
    captions = results_tb_clean_parsed, 
    index_path_map = meta_tb["index_path_map"]
    )
pprint(tender_file_extraction_final_tb)
cache.save_json('招标文件/tender_file_extraction_final_tb', tender_file_extraction_final_tb)

In [ ]:
#打印 增加型文档大纲框架 
from apps.projects.tiptap.helpers import TiptapUtils
enhanced_toc=TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
print(enhanced_toc)

### 1.4 附件提取和分析
（投标合规，提供评分的重要环节）

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final =cache.load_json('招标文件/tender_file_extraction_final')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(doc=tender_file_extraction_final, include_captions=False)
print(headings)

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb 输出：model_params, task, meta, formatted_message
# 核心功能模块： FindTopicContextBatch 里 prepare_context 调用 TiptapUtils.print_enhanced_toc(self.doc)

from apps.projects.services.tasks_preparation.appendix_templates.get_appendix_list import GetAppendixList
# #获取LLM分析相关数据
data_cooker_bid = GetAppendixList(tender_file_extraction_final)
model_params_bid, task_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t1-招标文件大纲分析/t1.5-附件分析/model_params_bid', model_params_bid)
cache.save_json('t1-招标文件大纲分析/t1.5-附件分析/tasks_bid', task_bid)
cache.save_json('t1-招标文件大纲分析/t1.5-附件分析/meta_bid', meta_bid)
cache.save_json('t1-招标文件大纲分析/t1.5-附件分析/formatted_messages_bid', formatted_messages_bid)

print(type(task_bid), len(task_bid))

In [ ]:
# LLM 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import clean_llm_JSON_output
analyzer = LLMService(model_params_bid)
results = await analyzer.process(task_bid)
results_clean_parsed = clean_llm_JSON_output(results)
pprint(results_clean_parsed)
cache.save_json('t1-招标文件大纲分析/t1.5-附件分析/results', results)
cache.save_json('t1-招标文件大纲分析/t1.5-附件分析/results_clean_parsed', results_clean_parsed)

### 1.5 附件响应

In [1]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final =cache.load_json('招标文件/tender_file_extraction_final')
appendix_list = cache.load_json('t1-招标文件大纲分析/t1.5-附件分析/results_clean_parsed')
paths = [item['path'] for item in appendix_list]
doc = cache.load_json('招标文件/tender_file_extraction_final')



In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb 输出：model_params, task, meta, formatted_message
# 核心功能模块： FindTopicContextBatch 里 prepare_context 调用 TiptapUtils.print_enhanced_toc(self.doc)

from apps.projects.services.tasks_preparation.appendix_templates.appendix_analysis import AppendixAnalysis
# #获取LLM分析相关数据
data_cooker_bid = AppendixAnalysis(tender_file_extraction_final, paths)
model_params_bid, tasks_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t1-招标文件大纲分析/t1.6-附件响应/model_params_bid', model_params_bid)
cache.save_json('t1-招标文件大纲分析/t1.6-附件响应/tasks_bid', tasks_bid)
cache.save_json('t1-招标文件大纲分析/t1.6-附件响应/meta_bid', meta_bid)
cache.save_json('t1-招标文件大纲分析/t1.6-附件响应/formatted_messages_bid', formatted_messages_bid)

print(type(tasks_bid), len(tasks_bid))

In [ ]:
pprint(formatted_messages_bid)

In [ ]:
print(len(tasks_bid[0]))
pprint(tasks_bid[0])

In [ ]:
print(tasks_bid[1]["context"])

In [ ]:
# LLM 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import clean_llm_JSON_output
system_role = "你是招投标领域的投标文件编写专家。"
analyzer = LLMService(model_params_bid)
results = await analyzer.process(tasks_bid[2])
# results_clean_parsed = clean_llm_JSON_output(results)
# cache.save_json('t1-招标文件大纲分析/t1.6-附件响应/results', results)
# cache.save_json('t1-招标文件大纲分析/t1.6-附件响应/results_clean_parsed', results_clean_parsed)
# pprint(results_clean_parsed)

In [ ]:

print(results)

In [ ]:
pprint(results_clean_parsed)

In [ ]:
# md table Display
from IPython.display import display, HTML, Markdown
display(Markdown(results))

## PART - 2. 确定和拆解写作任务

In [ ]:
## 工作流
# 任务线索： 评分细则，  资格审查， 投标文件编制要求， 附件要求，  项目需求 

#### <font color="red"> 1. 定位 线索 相关的章节</font>
输入： hints (List[str]) , doc(tiptapJSON) 输出：context_paths

In [ ]:
hints = ["评分细则", "资格审查/响应条款", "附件要求", "项目需求"]

## PART-2. 投标文件编辑格式和要求

#### <font color="red"> 1. 利用LLM定位相关的章节</font>
输入： tender_file_extraction_final_tb，topic_list 输出：List[relavant_parts]

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
print(headings)

In [2]:
# topics输入
topic_list = ["投标文件组成"]

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, topic_list 输出：model_params, task, meta, formatted_message
# 核心功能模块： FindTopicContextBatch 里 prepare_context 调用 TiptapUtils.print_enhanced_toc(self.doc)

from apps.projects.services.tasks_preparation.find_topic_context_batch import FindTopicContextBatch
# #获取LLM分析相关数据
data_cooker_bid = FindTopicContextBatch(tender_file_extraction_final_tb, topic_list)
model_params_bid, tasks_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/model_params_bid', model_params_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/tasks_bid', tasks_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/meta_bid', meta_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/formatted_messages_bid', formatted_messages_bid)

print(type(tasks_bid), len(tasks_bid))

In [ ]:
# LLM batch分析 (虽然这里实际只处理一个任务，但为了保持一致性，还是用batch)
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import process_concurrent_JSON_outputs
analyzer = LLMService(model_params_bid)
results = await analyzer.process_with_limit(tasks_bid, limit=5)
results_clean_parsed = process_concurrent_JSON_outputs(results)
pprint(results_clean_parsed)

# 清洗大模型输出结果，移除可能的Markdown代码块标记 并输出json.load格式
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/results', results)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/results_clean_parsed', results_clean_parsed)

#### <font color="red"> 2. 投标文件组成框架分析</font>
输入：tender_file_extraction_final_tb, List[relevant_parts]      
输出：list[]投标文件目录框架

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
relavant_parts = cache.load_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/results_clean_parsed')
paths = [item['path'] for item in relavant_parts]
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
pprint(paths)

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, paths 输出：model_params, task, meta, formatted_message
# 核心功能模块： BidOutlines 里 prepare_context 调用 TiptapUtils.extract_content_under_heading(self.data_input, path)
from apps.projects.services.tasks_preparation.bid_outlines import BidOutlines
# #获取LLM分析相关数据
data_cooker_bid = BidOutlines(tender_file_extraction_final_tb, paths)
model_params_bid, task_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/model_params_bid', model_params_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/tasks_bid', task_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/meta_bid', meta_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/formatted_messages_bid', formatted_messages_bid)

pprint(task_bid)

In [ ]:
# LLM 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import clean_llm_JSON_output
analyzer = LLMService(model_params_bid)
results = await analyzer.process(task_bid)
results_clean_parsed = clean_llm_JSON_output(results)
pprint(results_clean_parsed)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/results', results)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/results_clean_parsed', results_clean_parsed)

## PART-3. 投标人资格要求

#### <font color="red"> 1. 利用LLM定位相关的章节</font>
输入： tender_file_extraction_final_tb，topic_list 输出：List[relavant_parts]

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
print(headings)

In [2]:
# topics输入
topic_list = ["投标人资格要求", "资格要求相关的评标方法", "资格要求相关的模板"]

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, topic_list 输出：model_params, task, meta, formatted_message
# 核心功能模块： FindTopicContextBatch 里 prepare_context 调用 TiptapUtils.print_enhanced_toc(self.doc)

from apps.projects.services.tasks_preparation.find_topics_context import FindTopicsContext
# #获取LLM分析相关数据
data_cooker_bid = FindTopicsContext(tender_file_extraction_final_tb, topic_list)
model_params_bid, tasks_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/model_params_bid', model_params_bid)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/tasks_bid', tasks_bid)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/meta_bid', meta_bid)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/formatted_messages_bid', formatted_messages_bid)

print(type(tasks_bid), len(tasks_bid))

In [ ]:
# LLM batch分析 (虽然这里实际只处理一个任务，但为了保持一致性，还是用batch)
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import process_concurrent_JSON_outputs
analyzer = LLMService(model_params_bid)
results = await analyzer.process_with_limit([tasks_bid], limit=5)
results_clean_parsed = process_concurrent_JSON_outputs(results)
pprint(results_clean_parsed)

# 清洗大模型输出结果，移除可能的Markdown代码块标记 并输出json.load格式
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/results', results)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/results_clean_parsed', results_clean_parsed)

#### <font color="red"> 2. 投标人资格要求分析</font>
输入：tender_file_extraction_final_tb, List[relevant_parts]      
输出：md table

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
relavant_parts = cache.load_json('t3-投标人资格要求/t3.1-相关章节定位/results_clean_parsed')
raw_paths = [item['path'] for item in relavant_parts]
from apps.projects.services.task_service import deduplicate_paths
paths = deduplicate_paths(raw_paths)
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
pprint(paths)

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, paths 输出：model_params, task, meta, formatted_message
# 核心功能模块： BidOutlines 里 prepare_context 调用 TiptapUtils.extract_content_under_heading(self.data_input, path)
from apps.projects.services.tasks_preparation.bidder_qulification.qualification_analysis import QualificationAnalysis
# #获取LLM分析相关数据
data_cooker_bid = QualificationAnalysis(tender_file_extraction_final_tb, paths)
model_params_bid, task_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/model_params_bid', model_params_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/tasks_bid', task_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/meta_bid', meta_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/formatted_messages_bid', formatted_messages_bid)

pprint(task_bid)

In [ ]:
# LLM 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
analyzer = LLMService(model_params_bid)
results = await analyzer.process(task_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/results', results)

In [ ]:
# md table Display
from IPython.display import display, HTML, Markdown
display(Markdown(results))

#### <font color="red"> 3. 投标文件侧呈现</font>
输入：tender_file_extraction_final_tb, md_table, context      
输出：json 格式

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
Q_Req = cache.load_json('t3-投标人资格要求/t3.2-投标人资格要求分析/results')
task_bid = cache.load_json('t3-投标人资格要求/t3.2-投标人资格要求分析/tasks_bid')
supplement = task_bid['context']
print(supplement)

In [ ]:
# LLM 数据准备
# 输入： 之前分析的结果 Q_req (str), 之前分析的context作为supplement (str) 输出：model_params, task, meta, formatted_message
from apps.projects.services.tasks_preparation.bidder_qulification.bidding_preparation import BiddingPreparation
# #获取LLM分析相关数据
data_cooker_bid = BiddingPreparation(Q_Req, supplement)
model_params_bid, task_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/model_params_bid', model_params_bid)
cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/tasks_bid', task_bid)
cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/meta_bid', meta_bid)
cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/formatted_messages_bid', formatted_messages_bid)

pprint(task_bid)

In [ ]:
pprint(formatted_messages_bid)

In [ ]:
# LLM 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import clean_llm_JSON_output
analyzer = LLMService(model_params_bid)
results = await analyzer.process(task_bid)
results_clean_parsed = clean_llm_JSON_output(results)
cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/results', results)
cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/results_clean_parsed', results_clean_parsed)
pprint(results_clean_parsed)

In [ ]:
pprint(results_clean_parsed)

## 找到并使用模板

In [ ]:
Q_Req_sections = cache.load_json('t3-投标人资格要求/t3.3-投标文件呈现/results_clean_parsed')
print(type(Q_Req_sections))
pprint(Q_Req_sections)

In [13]:
secs_with_templates = [item for item in Q_Req_sections if item["已有模板"] != "无"]

In [ ]:
pprint(secs_with_templates)

## 注意： 待 前面添加 附件的单独处理， 附件为模板的， 专门用HTML处理，保持格式。